# Import des différentes librairies

In [8]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler #
from sklearn.preprocessing import MinMaxScaler #some normalization / standardization techniques
from sklearn.preprocessing import RobustScaler #
import matplotlib.pyplot as plt
import seaborn as sns
import pandas_profiling
from warnings import filterwarnings
filterwarnings("ignore")

In [9]:
train = pd.read_csv('data/train.csv')
test = pd.read_csv('data/test.csv')
threshold = train.shape[0]  #cette variable sera utile pour reséparer train et test plus tard.
target = np.log(train.SalePrice) #on met le prix au logarithme car la métrique utilisée sur Kaggle est le RMSLE.
print(threshold)
train.tail()

FileNotFoundError: [Errno 2] File b'data/train.csv' does not exist: b'data/train.csv'

# Préparation des données

Nous allons poster nos résultats sur Kaggle pour voir quelle place on peut avoir dans le leaderboard avec cette approche. 

Pour soumettre les résultats sur Kaggle, nous devons entraîner un modèle sur "train.csv", l'appliquer à "test.csv", puis mettre les prédictions dans un fichier csv (voir le fichier "sample_submission.csv"), qu'on uploadera sur kaggle sur ce lien : https://www.kaggle.com/c/house-prices-advanced-regression-techniques/submit.

On peut faire 10 soumissions par jour donc n'hésitez pas à en faire pour améliorer votre place sur le leaderboard !

On commence par append les dataframe train et test, car tout le feature engineering et le data cleaning qu'on fera sur le train, on doit aussi le faire sur le test pour avoir la même structure, en effet si on entraîne un modèle sur 10 features par exemple, on ne peut l'appliquer qu'avec ces 10 features, pas une de plus ni une de moins.


In [10]:
full_data = train.append(test)
full_data.Id = full_data.Id 
full_data = full_data.set_index('Id')



NameError: name 'test' is not defined

Je vous avais recommandé sur slack la librairie "pandas-profiling". <br>
C'est l'occasion de la voir à l'oeuvre !

In [11]:
full_data.profile_report(title='see how cool pandas_profiling is ?') 
#prend un petit peu de temps à charger car il y a beaucoup de colonnes

NameError: name 'full_data' is not defined

On remarque qu'on a pas mal de valeurs manquantes et beaucoup de zéros dans plusieurs variables. <br>
Les 50% de missing values de SalePrice sont tout à fait normaux : <br>
On a 1459 données d'entraînement et autant pour le test, hors on n'a pas le label dans le dataset de test. <br>
Normal, c'est ce qu'on cherche à prédire !

On a une correlation matrix faite automatiquement par pandas_profiling, commençons par appliquer un modèle très simple sur les variables les plus corrélées à la cible "SalePrice".

In [43]:
top_ten_features = full_data.corr()['SalePrice'].sort_values(ascending=False)[1:11]
print(top_ten_features)

OverallQual     0.790982
GrLivArea       0.708624
GarageCars      0.640409
GarageArea      0.623431
TotalBsmtSF     0.613581
1stFlrSF        0.605852
FullBath        0.560664
TotRmsAbvGrd    0.533723
YearBuilt       0.522897
YearRemodAdd    0.507101
Name: SalePrice, dtype: float64


Essayons déjà de nettoyer un peu ces 10 colonnes et d'appliquer un modèle simple (regression linéaire), peut-être que le résultat sera déjà satisfaisant.

<h3> OverallQual </h3>

La colonne ne contient pas de valeurs manquantes, pas de travail particulier à réaliser dessus pour le moment.
Peut-être essayer de réduire l'amplitude des notes (1-10) en 1-5 ou 1-3 améliorera le modèle, à tester.

<h3> GrLivArea </h3>

Pas de NaN, c'est ok de ce côté là.
Comme les valeurs sont assez réparties, on va essayer de normaliser la colonne.

<h3> GarageCars et GarageArea </h3>

1 seule valeur NaN qu'on va corriger, ici il faut faire un choix entre les deux features, pour pas donner trop de poids au garage dans le modèle (on répète l'information) et pour ma part je privilégie GarageArea car on pourra la normaliser après.<br>
Les 5.4% de valeurs 0 sont dues au fait que certaines maisons n'ont pas de garage.

In [44]:
full_data.GarageArea.fillna(full_data.GarageArea.median());

<h3>TotalBsmtSF</h3>

Pareil, une seule valeur NaN à corriger, et des 0 qui sont dûs au fait que toutes les maisons n'ont pas forcément de sous-sol.

In [45]:
full_data.TotalBsmtSF.fillna(full_data.TotalBsmtSF.median());

<h3>1stFloorSF</h3>

Pas de valeur NaN, a priori pas grand chose à faire si ce n'est de la normalisation.

<h3> FullBath et TotRmsAbvGrd</h3>

A priori rien à faire.

<h3>YearBuilt et YearRemodAdd</h3>

Pas de NaN, ce qui pourrait être intéressant ici serait de faire du feature engineering pour créer des catégories, par exemple maison construite entre 1892 et 1930, 1930 et 1970, 1970 et 2010, et donc réduire l'effet d'éparpillement, ce qui devrait faciliter l'apprentissage du modèle (à tester)

Pour YearRemodAdd le min est à 1950 donc peut-être 1950-1970, 1970-1990, 1990-2010 ... 

In [46]:
selected_features = list(dict(top_ten_features)) 
   
    #on transforme la série en dictionnaire,
    #et en appliquant la fonction list on récupère la liste des clés du dictionnaires.

selected_features.pop(2) #on ne garde pas la colonne GarageCars
selected_features

['OverallQual',
 'GrLivArea',
 'GarageArea',
 'TotalBsmtSF',
 '1stFlrSF',
 'FullBath',
 'TotRmsAbvGrd',
 'YearBuilt',
 'YearRemodAdd']

On a eu de la chance, les 10 features les plus correllées étaient très simple à nettoyer, même si on n'en garde que 9 au final.

Essayons un premier modèle : la regression linéaire sur ces 9 features.

In [47]:
train = full_data[:threshold].drop('SalePrice', axis = 1) #voilà pourquoi on a stocké la variable threshold plus haut.


In [48]:
train.columns #on enlève la variable à prédire du dataset

Index(['1stFlrSF', '2ndFlrSF', '3SsnPorch', 'Alley', 'BedroomAbvGr',
       'BldgType', 'BsmtCond', 'BsmtExposure', 'BsmtFinSF1', 'BsmtFinSF2',
       'BsmtFinType1', 'BsmtFinType2', 'BsmtFullBath', 'BsmtHalfBath',
       'BsmtQual', 'BsmtUnfSF', 'CentralAir', 'Condition1', 'Condition2',
       'Electrical', 'EnclosedPorch', 'ExterCond', 'ExterQual', 'Exterior1st',
       'Exterior2nd', 'Fence', 'FireplaceQu', 'Fireplaces', 'Foundation',
       'FullBath', 'Functional', 'GarageArea', 'GarageCars', 'GarageCond',
       'GarageFinish', 'GarageQual', 'GarageType', 'GarageYrBlt', 'GrLivArea',
       'HalfBath', 'Heating', 'HeatingQC', 'HouseStyle', 'KitchenAbvGr',
       'KitchenQual', 'LandContour', 'LandSlope', 'LotArea', 'LotConfig',
       'LotFrontage', 'LotShape', 'LowQualFinSF', 'MSSubClass', 'MSZoning',
       'MasVnrArea', 'MasVnrType', 'MiscFeature', 'MiscVal', 'MoSold',
       'Neighborhood', 'OpenPorchSF', 'OverallCond', 'OverallQual',
       'PavedDrive', 'PoolArea', 'PoolQC',

In [49]:
selected_features_df = train[selected_features]


In [50]:
X_train, X_test, Y_train, Y_test = train_test_split(selected_features_df, target, test_size = 0.2, random_state = 1337)


In [51]:
first_model = LinearRegression()

In [52]:
first_model.fit(X_train, Y_train)
prices_predicted = np.exp(first_model.predict(X_test))

In [13]:
sns.scatterplot(range(X_test.shape[0]), np.exp(Y_test))
sns.scatterplot(range(X_test.shape[0]), prices_predicted)

NameError: name 'X_test' is not defined

## La prédiction n'est pas mauvaise !

Calculons maintenant le R squared

In [54]:
first_model.score(X_test, Y_test)

0.8218556379121128

Pas mal..., essayons de voir sur Kaggle où on se place sur le leaderboard.

In [63]:
selected_features_test_df.profile_report()

In [67]:
selected_features_test_df = selected_features_test_df.fillna(0)

In [68]:
#selected_features_test_df = test[selected_features]
first_model.predict(selected_features_test_df)

array([11.7212606 , 11.86843238, 11.97934736, ..., 11.90399315,
       11.72654049, 12.33693195])

In [69]:
submission = pd.read_csv('data/sample_submission.csv')

In [71]:
submission.SalePrice = first_model.predict(selected_features_test_df)

In [73]:
submission.to_csv('data/premier_test.csv', index=False)